**Dataclass** is a class that typically contains only Class. There are no restrictions though

In [1]:
from dataclasses import dataclass

@dataclass
class DataClassCard:
    rank: str
    suit: str

In [3]:
qoh = DataClassCard('Q','hearts')
print(qoh.rank)
print(qoh)

Q
DataClassCard(rank='Q', suit='hearts')


In [6]:
class RegCard:
    def __init__(self, rank, suit) -> None:
        self.rank = rank
        self.suit = suit
    #The below methods are built by default in Dataclass
    def __repr__(self):
        return (f'{self.__class__.__name__}'
                f'(rank={self.rank!r}, suit={self.suit!r})')

    def __eq__(self, other):
        if other.__class__ is not self.__class__:
            return NotImplemented
        return (self.rank, self.suit) == (other.rank, other.suit)

In [5]:
rqoh = RegCard('Q','Spade')
print(rqoh)

How to add default values to data class fields

How data classes allow for ordering of objects

How to represent immutable data

How data classes handle inheritance

In [11]:
# More suitable alternate will be Namedtuple
from collections import namedtuple

nmecard = namedtuple('NameTupcard',['rank','suit'])

In [15]:
nmecard('Q', 'clubs') ## the named tuple creates an object which then acts as template

## Cannot use the NameTupCard to instantiate the object, need to use nmecard variable
# There are inherent restriction like the NamedTuple is immutable, and no default value 

NameTupcard(rank='Q', suit='clubs')

In addition to tuple, dict, namedtuple, and attrs, there are many other similar projects, including typing.NamedTuple, namedlist, attrdict, plumber, and fields. While data classes are a great new alternative.

In [16]:
@dataclass
class Position:
    name: str
    lon: float
    lat: float

In [18]:
pos = Position('oslo',58,56)
pos

Position(name='oslo', lon=58, lat=56)

In [19]:
from dataclasses import make_dataclass

Position = make_dataclass('Position', ['name', 'lat', 'lon'])

In [20]:
@dataclass
class Position:
    name: str
    lon: float = 0.0
    lat: float = 0.0

In [21]:
Position('Green Valley')

Position(name='Green Valley', lon=0.0, lat=0.0)

In [22]:
#Type of the attributes is mandatory, however if you don't want 
# provide a type, then better use Any

from dataclasses import dataclass
from typing import Any

@dataclass
class WithoutExplicitTypes:
    name: Any
    value: Any = 42

In [23]:
from math import asin, cos, radians, sin, sqrt

@dataclass
class Position:
    name: str
    lon: float = 0.0
    lat: float = 0.0

    def distance_to(self, other):
        r = 6371  # Earth radius in kilometers
        lam_1, lam_2 = radians(self.lon), radians(other.lon)
        phi_1, phi_2 = radians(self.lat), radians(other.lat)
        h = (sin((phi_2 - phi_1) / 2)**2
             + cos(phi_1) * cos(phi_2) * sin((lam_2 - lam_1) / 2)**2)
        return 2 * r * asin(sqrt(h))

In [24]:
os = Position('osls',57.8,26.8)
ds = Position('denves',80.8,76.8)
os.distance_to(ds)

5693.336554050493

about some more advanced features **like parameters to** the @dataclass decorator and the **field()** function. Together, they give you **more control** when creating a data class.

In [25]:
from typing import List

@dataclass
class PlayingCard:
    rank: str
    suit: str

@dataclass
class Deck:
    cards: List[PlayingCard]

In [26]:
koh = PlayingCard('k','spade')
qos = PlayingCard('q','spade')

two = Deck([koh, qos])

In [27]:
two

Deck(cards=[PlayingCard(rank='k', suit='spade'), PlayingCard(rank='q', suit='spade')])

In [4]:
RANKS = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
SUITS = '♣ ♢ ♡ ♠'.split()

def make_french_deck():
    return [PlayingCard(r, s) for s in SUITS for r in RANKS]

In [31]:
deck = make_french_deck()

In [32]:
type(deck)

list

In [33]:
@dataclass
class Deck:  # Will NOT work
    cards: List[PlayingCard] = make_french_deck()

ValueError: mutable default <class 'list'> for field cards is not allowed: use default_factory

In [38]:
from dataclasses import field

@dataclass
class Deck:
    # using the field function to create the elements 
    cards: List[PlayingCard] = field(default_factory=make_french_deck)

    def __repr__(self):
        cards = ', '.join(f'{c!s}' for c in self.cards)
        return f'{self.__class__.__name__}({cards})'

default: Default value of the field

default_factory: Function that returns the initial value of the field

init: Use field in .__init__() method? (Default is True.)

repr: Use field in repr of the object? (Default is True.)

compare: Include the field in comparisons? (Default is True.)

hash: Include the field when calculating hash()? (Default is to use the same as for compare.)

metadata: A mapping with information about the field

In [40]:
deckedlist = Deck()
deckedlist

#The below representation possible due to the __repr__ method added above
# Note the !s specifier in the {c!s} format string. It means 
# that we explicitly want to use the str() representation of each PlayingCard.

Deck(♣2, ♣3, ♣4, ♣5, ♣6, ♣7, ♣8, ♣9, ♣10, ♣J, ♣Q, ♣K, ♣A, ♢2, ♢3, ♢4, ♢5, ♢6, ♢7, ♢8, ♢9, ♢10, ♢J, ♢Q, ♢K, ♢A, ♡2, ♡3, ♡4, ♡5, ♡6, ♡7, ♡8, ♡9, ♡10, ♡J, ♡Q, ♡K, ♡A, ♠2, ♠3, ♠4, ♠5, ♠6, ♠7, ♠8, ♠9, ♠10, ♠J, ♠Q, ♠K, ♠A)

In [36]:
@dataclass
class Position:
    name: str
    lon: float = field(default=0.0, metadata={'unit': 'degrees'})
    lat: float = field(default=0.0, metadata={'unit': 'degrees'})

In [37]:
@dataclass
class PlayingCard:
    rank: str
    suit: str

    def __str__(self):
        return f'{self.suit}{self.rank}'

Parameters that can be added to dataclass decorator

init: Add .__init__() method? (Default is True.)

repr: Add .__repr__() method? (Default is True.)

eq: Add .__eq__() method? (Default is True.)

order: Add ordering methods? (Default is False.)

unsafe_hash: Force the addition of a .__hash__() method? (Default is False.)

frozen: If True, assigning to fields raise an exception. (Default is False.)

In [2]:
from dataclasses import dataclass

@dataclass(order=True)
class PlayingCard:
    rank: str
    suit: str

    def __str__(self):
        return f'{self.suit}{self.rank}'

In [5]:
card = PlayingCard('Q', '♡')

RANKS.index(card.rank) * len(SUITS) + SUITS.index(card.suit)

42

In [6]:
card

PlayingCard(rank='Q', suit='♡')

In [44]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Position:
    name: str
    lon: float = 0.0
    lat: float = 0.0

In [45]:
poslo = Position('oslo',10,86)

In [46]:
poslo

Position(name='oslo', lon=10, lat=86)

In [47]:
poslo.lon = 6

FrozenInstanceError: cannot assign to field 'lon'

In [48]:
@dataclass(frozen=True)
class ImmutableCard:
    rank: str
    suit: str

@dataclass(frozen=True)
class ImmutableDeck:
    """This is still mutable"""
    cards: List[ImmutableCard]

In [49]:
#Inheritance... Simple

@dataclass
class Position:
    name: str
    lon: float
    lat: float # No default value

@dataclass
class Capital(Position):
    country: str

In [50]:
#Inheritance ... complex

@dataclass
class Position:
    name: str
    lon: float = 0.0
    lat: float = 2.0

@dataclass
class Capital(Position):
    country: str

# this is not valid Python. If a parameter has a default value, all following parameters must also have a default value. In other words, if a field in a base class has a default value, then all new fields added in a subclass must have default values as well.

TypeError: non-default argument 'country' follows default argument

In [51]:
@dataclass
class SimplePosition:
    name: str
    lon: float
    lat: float

@dataclass
class SlotPosition:
    __slots__ = ['name', 'lon', 'lat']
    name: str
    lon: float
    lat: float

In [52]:
from pympler import asizeof
simple = SimplePosition('London', -0.1, 51.5)
slot = SlotPosition('Madrid', -3.7, 40.4)
asizeof.asizesof(simple, slot)

(624, 160)